# Introducción a TensorFlow

¡Bienvenidos a la tarea de programación de esta semana! Hasta ahora, siempre has utilizado Numpy para construir redes neuronales, pero esta semana explorarás un marco de aprendizaje profundo que te permite construir redes neuronales más fácilmente. Los marcos de aprendizaje automático como TensorFlow, PaddlePaddle, Torch, Caffe, Keras y muchos otros pueden acelerar su desarrollo de aprendizaje automático significativamente. TensorFlow 2.3 ha introducido mejoras significativas con respecto a su predecesor, algunas de las cuales encontrará e implementará aquí.

Al final de esta tarea, usted será capaz de hacer lo siguiente en TensorFlow 2.3:

* Utilizar `tf.Variable` para modificar el estado de una variable
* Explicar la diferencia entre una variable y una constante
* Entrenar una red neuronal en un conjunto de datos TensorFlow

Los marcos de programación como TensorFlow no sólo reducen el tiempo de codificación, sino que también pueden realizar optimizaciones que aceleran el propio código. 

## Nota importante sobre el envío al AutoGrader

Antes de enviar su tarea al AutoGrader, asegúrese de que no está haciendo lo siguiente:

1. No ha añadido ninguna declaración _extra_ `print` en la tarea.
2. No ha añadido ninguna celda de código _extra_ en la tarea.
3. No ha cambiado ningún parámetro de la función.
4. No ha utilizado ninguna variable global dentro de sus ejercicios calificados. A menos que se le indique específicamente que lo haga, por favor absténgase de hacerlo y utilice las variables locales en su lugar.
5. No está cambiando el código de asignación donde no es necesario, como la creación de variables _extra_.

Si hace algo de lo siguiente, obtendrá un error como `Grader no encontrado` (o similarmente inesperado) al enviar su tarea. Antes de pedir ayuda/depurar los errores de su tarea, compruebe esto primero. Si este es el caso, y no recuerda los cambios que ha realizado, puede obtener una nueva copia de la tarea siguiendo estas [instrucciones](https://www.coursera.org/learn/deep-neural-network/supplement/QWEnZ/h-ow-to-refresh-your-workspace).

## Table of Contents
- [1- Packages](#1)
    - [1.1 - Checking TensorFlow Version](#1-1)
- [2 - Basic Optimization with GradientTape](#2)
    - [2.1 - Linear Function](#2-1)
        - [Exercise 1 - linear_function](#ex-1)
    - [2.2 - Computing the Sigmoid](#2-2)
        - [Exercise 2 - sigmoid](#ex-2)
    - [2.3 - Using One Hot Encodings](#2-3)
        - [Exercise 3 - one_hot_matrix](#ex-3)
    - [2.4 - Initialize the Parameters](#2-4)
        - [Exercise 4 - initialize_parameters](#ex-4)
- [3 - Building Your First Neural Network in TensorFlow](#3)
    - [3.1 - Implement Forward Propagation](#3-1)
        - [Exercise 5 - forward_propagation](#ex-5)
    - [3.2 Compute the Cost](#3-2)
        - [Exercise 6 - compute_cost](#ex-6)
    - [3.3 - Train the Model](#3-3)
- [4 - Bibliography](#4)

<a name='1'></a>
## 1 - Packages

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.python.ops.resource_variable_ops import ResourceVariable
import time

<a name='1-1'></a>
### 1.1 - Comprobación de la versión de TensorFlow 

Utilizarás la versión 2.3 para esta tarea, para obtener la máxima velocidad y eficiencia.

In [ ]:
tf.__version__

<a name='2'></a>
## 2 - Optimización básica con GradientTape

La belleza de TensorFlow 2 está en su simplicidad. Básicamente, todo lo que necesitas hacer es implementar la propagación hacia adelante a través de un gráfico computacional. TensorFlow calculará las derivadas por ti, moviéndose hacia atrás a través del gráfico registrado con `GradientTape`. Lo único que te queda por hacer es especificar la función de coste y el optimizador que quieres utilizar. 

Al escribir un programa TensorFlow, el objeto principal que se utiliza y transforma es el `tf.Tensor`. Estos tensores son el equivalente en TensorFlow a los arrays de Numpy, es decir, arrays multidimensionales de un tipo de datos determinado que también contienen información sobre el gráfico computacional.

A continuación, utilizarás `tf.Variable` para almacenar el estado de tus variables. Las variables sólo pueden crearse una vez, ya que su valor inicial define la forma y el tipo de la variable. Además, el arg `dtype` en `tf.Variable` puede ser establecido para permitir que los datos sean convertidos a ese tipo. Pero si no se especifica ninguno, se mantendrá el tipo de datos si el valor inicial es un tensor, o lo decidirá `convert_to_tensor`. Generalmente es mejor que se especifique directamente, ¡así no se rompe nada!


Aquí llamarás al conjunto de datos TensorFlow creado en un archivo HDF5, que puedes utilizar en lugar de un array Numpy para almacenar tus conjuntos de datos. ¡Puedes pensar en esto como un generador de datos TensorFlow! 

Utilizarás el conjunto de datos Hand sign, que está compuesto por imágenes con forma 64x64x3.

In [ ]:
train_dataset = h5py.File('datasets/train_signs.h5', "r")
test_dataset  = h5py.File('datasets/test_signs.h5', "r")

In [ ]:
x_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_x'])
y_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_y'])

x_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_x'])
y_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_y'])

In [ ]:
type(x_train)

Dado que los conjuntos de datos de TensorFlow son generadores, no puedes acceder directamente a su contenido a menos que iteres sobre ellos en un bucle for, o creando explícitamente un iterador de Python usando `iter` y consumiendo sus
elementos con `next`. Además, puedes inspeccionar la "forma" y el "tipo" de cada elemento utilizando el atributo "element_spec".

In [ ]:
print(x_train.element_spec)

In [ ]:
print(next(iter(x_train)))

El conjunto de datos que utilizarás durante esta tarea es un subconjunto de los dígitos del lenguaje de signos. Contiene seis clases diferentes que representan los dígitos del 0 al 5.

In [ ]:
unique_labels = set()
for element in y_train:
    unique_labels.add(element.numpy())
print(unique_labels)

Puede ver algunas de las imágenes del conjunto de datos ejecutando la siguiente celda.

In [ ]:
images_iter = iter(x_train)
labels_iter = iter(y_train)
plt.figure(figsize=(10, 10))
for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(next(images_iter).numpy().astype("uint8"))
    plt.title(next(labels_iter).numpy().astype("uint8"))
    plt.axis("off")

Hay una diferencia adicional entre los conjuntos de datos de TensorFlow y las matrices de Numpy: Si necesitas transformar uno, invocarías el método `map` para aplicar la función pasada como argumento a cada uno de los elementos.

In [ ]:
def normalize(image):
    """
    Transformar una imagen en un tensor de forma (64 * 64 * 3, )
    y normalizar sus componentes.
    
    Argumentos
    imagen - Tensor.
    
    Devuelve 
    resultado -- Tensor transformado 
    """
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [-1,])
    return image

In [ ]:
new_train = x_train.map(normalize)
new_test = x_test.map(normalize)

In [ ]:
new_train.element_spec

In [ ]:
print(next(iter(new_train)))

<a name='2-1'></a>
### 2.1 - Función lineal

Comencemos este ejercicio de programación calculando la siguiente ecuación: $Y = WX + b$, donde $W$ y $X$ son matrices aleatorias y b es un vector aleatorio. 

<a name='ex-1'></a>
### Exercise 1 - linear_function

Calcule $WX + b$ donde $W, X$ y $b$ se extraen de una distribución normal aleatoria. W tiene forma (4, 3), X es (3,1) y b es (4,1). A modo de ejemplo, así se define una constante X con la forma (3,1):
```python
X = tf.constant(np.random.randn(3,1), name = "X")

```
Ten en cuenta que la diferencia entre `tf.constant` y `tf.Variable` es que puedes modificar el estado de una `tf.Variable` pero no puedes cambiar el estado de una `tf.constant`.
```

Las siguientes funciones pueden resultarle útiles: 
- tf.matmul(..., ...) para hacer una multiplicación de matrices
- tf.add(..., ...) para hacer una suma
- np.random.randn(...) para inicializar aleatoriamente
    
<p hidden> 
    X = tf.constant(np.random.randn(3,1), name = "X")
    W = tf.constant(np.random.randn(4,3), name = "w")
    b = tf.constant(np.random.randn(4,1), name = "b")
    Y = tf.add( tf.matmul ( W, X ), b) 
</p> 

In [ ]:
# GRADED FUNCTION: linear_function

def linear_function():
    """
    Implements a linear function: 
            Initializes X to be a random tensor of shape (3,1)
            Initializes W to be a random tensor of shape (4,3)
            Initializes b to be a random tensor of shape (4,1)
    Returns: 
    result -- Y = WX + b 
    """

    np.random.seed(1)
    
    """
    Note, to ensure that the "random" numbers generated match the expected results,
    please create the variables in the order given in the starting code below.
    (Do not re-arrange the order).
    Nota, para asegurar que los números "aleatorios" generados coinciden con los resultados esperados,
    por favor, cree las variables en el orden que se indica en el código de inicio.
    (No reordene el orden).
    """
    # (approx. 4 lines)
    # X = ...
    # W = ...
    # b = ...
    # Y = ...
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    return Y

In [ ]:
result = linear_function()
print(result)

assert type(result) == EagerTensor, "Use the TensorFlow API"
assert np.allclose(result, [[-2.15657382], [ 2.95891446], [-1.08926781], [-0.84538042]]), "Error"
print("\033[92mAll test passed")


**Expected Output**: 

```
result = 
[[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]]
```

<a name='2-2'></a>
### 2.2 - Cálculo de la sigmoidea 
Increíble. Acabas de implementar una función lineal. TensorFlow ofrece una variedad de funciones de redes neuronales de uso común como `tf.sigmoid` y `tf.softmax`.

Para este ejercicio, calcula el sigmoide de z. 

En este ejercicio, usted: Convierte tu tensor en el tipo `float32` usando `tf.cast`, luego calcula el sigmoide usando `tf.keras.activations.sigmoid`. 

<a name='ex-2'></a>
### Exercise 2 - sigmoid

Implementa la función sigmoidea que aparece a continuación. Debe utilizar lo siguiente: 

- `tf.cast("...", tf.float32)`
- `tf.keras.activations.sigmoid("...")`

<p hidden>
z = tf.cast(z, tf.float32)
a = tf.keras.activations.sigmoid(z)
</p>

In [ ]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    
    """
    Calcula la sigmoidea de z
    
    Argumentos:
    z -- valor de entrada, escalar o vector
    
    Devuelve: 
    a -- (tf.float32) la sigmoide de z
    """
    # tf.keras.activations.sigmoid requires float16, float32, float64, complex64, or complex128.
    
    # (approx. 2 lines)
    # z = ...z = tf.cast(z, tf.float32)
    # a = tf.keras.activations.sigmoid(z)
    # a = ...
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    return a
# z = tf.cast(z, tf.float32)
#     a = tf.keras.activations.sigmoid(z)

In [ ]:
result = sigmoid(-1)
print ("type: " + str(type(result)))
print ("dtype: " + str(result.dtype))
print ("sigmoid(-1) = " + str(result))
print ("sigmoid(0) = " + str(sigmoid(0.0)))
print ("sigmoid(12) = " + str(sigmoid(12)))

def sigmoid_test(target):
    result = target(0)
    assert(type(result) == EagerTensor)
    assert (result.dtype == tf.float32)
    assert sigmoid(0) == 0.5, "Error"
    assert sigmoid(-1) == 0.26894143, "Error"
    assert sigmoid(12) == 0.9999939, "Error"

    print("\033[92mAll test passed")

sigmoid_test(sigmoid)

**Expected Output**: 
<table>
<tr> 
<td>
type
</td>
<td>
class 'tensorflow.python.framework.ops.EagerTensor'
</td>
</tr><tr> 
<td>
dtype
</td>
<td>
"dtype: 'float32'
</td>
</tr>
<tr> 
<td>
Sigmoid(-1)
</td>
<td>
0.2689414
</td>
</tr>
<tr> 
<td>
Sigmoid(0)
</td>
<td>
0.5
</td>
</tr>
<tr> 
<td>
Sigmoid(12)
</td>
<td>
0.999994
</td>
</tr> 

</table> 

<a name='2-3'></a>
### 2.3 - Uso de codificaciones en caliente

Muchas veces en el aprendizaje profundo tendrás un vector $Y$ con números que van de $0$ a $C-1$, donde $C$ es el número de clases. Si $C$ es, por ejemplo, 4, entonces podrías tener el siguiente vector y que tendrás que convertir así:


<img src="images/onehot.png" style="width:600px;height:150px;">

Esto se llama codificación "one hot", porque en la representación convertida, exactamente un elemento de cada columna es "hot" (lo que significa que se establece en 1). Para hacer esta conversión en numpy, puede que tengas que escribir unas cuantas líneas de código. En TensorFlow, puedes usar una línea de código: 

- [tf.one_hot(labels, depth, axis=0)](https://www.tensorflow.org/api_docs/python/tf/one_hot)

`axis=0` indica que el nuevo eje (axis)  se crea en la dimensión 0

<a name='ex-3'></a>
### Exercise 3 - one_hot_matrix

Implemente la función de abajo para tomar una etiqueta y el número total de clases $C$, y devuelva la codificación de un caliente en una matriz de columnas. ¡Utilice `tf.one_hot()` para hacer esto, y `tf.reshape()` para remodelar su tensor one hot! 

- `tf.reshape(tensor, shape)`
<p hidden>tf.reshape(tf.one_hot(label, depth, axis=0),shape=(-1,))</p>

In [ ]:
# GRADED FUNCTION: one_hot_matrix
def one_hot_matrix(label, depth=6):
    """
    Calcula la codificación en caliente de una sola etiqueta
    
    Argumentos:
        label -- (int) Etiquetas categóricas
        depth -- (int) Número de clases diferentes que puede tomar la etiqueta
    
    Devuelve:
         one_hot -- tf.Tensor Una matriz de una columna con la codificación one hot.
    """
    # (approx. 1 line)
    # one_hot =...
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    return one_hot

In [ ]:
def one_hot_matrix_test(target):
    label = tf.constant(1)
    depth = 4
    result = target(label, depth)
    print("Test 1:",result)
    assert result.shape[0] == depth, "Use the parameter depth"
    assert np.allclose(result, [0., 1. ,0., 0.] ), "Wrong output. Use tf.one_hot"
    label_2 = [2]
    result = target(label_2, depth)
    print("Test 2:", result)
    assert result.shape[0] == depth, "Use the parameter depth"
    assert np.allclose(result, [0., 0. ,1., 0.] ), "Wrong output. Use tf.reshape as instructed"
    
    print("\033[92mAll test passed")

one_hot_matrix_test(one_hot_matrix)

**Expected output**
```
Test 1: tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
Test 2: tf.Tensor([0. 0. 1. 0.], shape=(4,), dtype=float32)
```

In [ ]:
new_y_test = y_test.map(one_hot_matrix)
new_y_train = y_train.map(one_hot_matrix)

In [ ]:
print(next(iter(new_y_test)))

<a name='2-4'></a>
### 2.4 - Inicializar los parámetros 

Ahora vas a inicializar un vector de números con el inicializador Glorot. La función a la que llamarás es `tf.keras.initializers.GlorotNormal`, que extrae muestras de una distribución normal truncada centrada en 0, con `stddev = sqrt(2 / (fan_in + fan_out))`, donde `fan_in` es el número de unidades de entrada y `fan_out` es el número de unidades de salida, ambas en el tensor de pesos. 

Para inicializar con ceros o unos puedes usar `tf.zeros()` o `tf.ones()` en su lugar. 

<a name='ex-4'></a>
### Exercise 4 - initialize_parameters

Implementa la función de abajo para tomar una forma y devolver un array de números usando el inicializador GlorotNormal. 


 - `tf.keras.initializers.GlorotNormal(seed=1)`
 - `tf.Variable(initializer(shape=())`
 
<P hidden>
    W1 = tf.Variable(initializer(shape=(25, 12288)))
    b1 = tf.Variable(initializer(shape=(25, 1)))
    W2 = tf.Variable(initializer(shape=(12, 25)))
    b2 = tf.Variable(initializer(shape=(12, 1)))
    W3 = tf.Variable(initializer(shape=(6, 12)))
    b3 = tf.Variable(initializer(shape=(6, 1)))
<\p>

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters():
    """
    Initializes parameters to build a neural network with TensorFlow. The shapes are:
                        W1 : [25, 12288]
                        b1 : [25, 1]
                        W2 : [12, 25]
                        b2 : [12, 1]
                        W3 : [6, 12]
                        b3 : [6, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
                                
    initializer = tf.keras.initializers.GlorotNormal(seed=1)   
    #(approx. 6 lines of code)
    
    
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [ ]:
def initialize_parameters_test(target):
    parameters = target()

    values = {"W1": (25, 12288),
              "b1": (25, 1),
              "W2": (12, 25),
              "b2": (12, 1),
              "W3": (6, 12),
              "b3": (6, 1)}

    for key in parameters:
        print(f"{key} shape: {tuple(parameters[key].shape)}")
        assert type(parameters[key]) == ResourceVariable, "All parameter must be created using tf.Variable"
        assert tuple(parameters[key].shape) == values[key], f"{key}: wrong shape"
        assert np.abs(np.mean(parameters[key].numpy())) < 0.5,  f"{key}: Use the GlorotNormal initializer"
        assert np.std(parameters[key].numpy()) > 0 and np.std(parameters[key].numpy()) < 1, f"{key}: Use the GlorotNormal initializer"

    print("\033[92mAll test passed")
    
initialize_parameters_test(initialize_parameters)

**Expected output**
```
W1 shape: (25, 12288)
b1 shape: (25, 1)
W2 shape: (12, 25)
b2 shape: (12, 1)
W3 shape: (6, 12)
b3 shape: (6, 1)
```

In [ ]:
parameters = initialize_parameters()

<a name='3'></a>
## 3 - Building Your First Neural Network in TensorFlow

En esta parte de la tarea construirás una red neuronal utilizando TensorFlow. Recuerda que hay dos partes para implementar un modelo TensorFlow:

- Implementar la propagación hacia adelante
- Recuperar los gradientes y entrenar el modelo

¡Vamos a ponernos manos a la obra!

<a name='3-1'></a>
### 3.1 - Implement Forward Propagation 

Uno de los grandes puntos fuertes de TensorFlow radica en el hecho de que sólo tienes que implementar la función de propagación hacia adelante y él llevará la cuenta de las operaciones que hiciste para calcular la propagación hacia atrás automáticamente.  


<a name='ex-5'></a>
### Exercise 5 - forward_propagation

Implementar la función `forward_propagation`.

**Nota** Utilizar sólo la API TF. 

- tf.math.add
- tf.linalg.matmul
- tf.keras.activations.relu

<p hidden>
Z1 = tf.math.add(tf.linalg.matmul(W1,X),b1)    
A1 = tf.keras.activations.relu(Z1)          
Z2 = tf.math.add(tf.linalg.matmul(W2,A1),b2)
A2 = tf.keras.activations.relu(Z2)          
Z3 = tf.math.add(tf.linalg.matmul(W3,A2),b3)
<\p>

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Implementa la propagación hacia adelante para el modelo LINEAL -> RELU -> LINEAL -> RELU -> LINEAL
    
    Argumentos:
    X -- marcador de entrada del conjunto de datos, de forma (tamaño de entrada, número de ejemplos)
    parameters -- diccionario python que contiene sus parámetros "W1", "b1", "W2", "b2", "W3", "b3"
                  las formas se dan en initialize_parameters

    Devuelve:
    Z3 -- la salida de la última unidad LINEAL
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    #(approx. 5 lines)                   # Numpy Equivalents:
    # Z1 = ...                           # Z1 = np.dot(W1, X) + b1
    # A1 = ...                           # A1 = relu(Z1)
    # Z2 = ...                           # Z2 = np.dot(W2, A1) + b2
    # A2 = ...                           # A2 = relu(Z2)
    # Z3 = ...                           # Z3 = np.dot(W3, A2) + b3
    # YOUR CODE STARTS HERE
    
    
    
    # YOUR CODE ENDS HERE
    
    return Z3

In [ ]:
def forward_propagation_test(target, examples):
    minibatches = examples.batch(2)
    parametersk = initialize_parameters()
    W1 = parametersk['W1']
    b1 = parametersk['b1']
    W2 = parametersk['W2']
    b2 = parametersk['b2']
    W3 = parametersk['W3']
    b3 = parametersk['b3']
    index = 0
    minibatch = list(minibatches)[0]
    with tf.GradientTape() as tape:
        forward_pass = target(tf.transpose(minibatch), parametersk)
        print(forward_pass)
        fake_cost = tf.reduce_mean(forward_pass - np.ones((6,2)))

        assert type(forward_pass) == EagerTensor, "Your output is not a tensor"
        assert forward_pass.shape == (6, 2), "Last layer must use W3 and b3"
        assert np.allclose(forward_pass, 
                           [[-0.13430887,  0.14086473],
                            [ 0.21588647, -0.02582335],
                            [ 0.7059658,   0.6484556 ],
                            [-1.1260961,  -0.9329492 ],
                            [-0.20181894, -0.3382722 ],
                            [ 0.9558965,   0.94167566]]), "Output does not match"
    index = index + 1
    trainable_variables = [W1, b1, W2, b2, W3, b3]
    grads = tape.gradient(fake_cost, trainable_variables)
    assert not(None in grads), "Wrong gradients. It could be due to the use of tf.Variable whithin forward_propagation"
    print("\033[92mAll test passed")

forward_propagation_test(forward_propagation, new_train)

**Expected output**
```
tf.Tensor(
[[-0.13430887  0.14086473]
 [ 0.21588647 -0.02582335]
 [ 0.7059658   0.6484556 ]
 [-1.1260961  -0.9329492 ]
 [-0.20181894 -0.3382722 ]
 [ 0.9558965   0.94167566]], shape=(6, 2), dtype=float32)
```

<a name='3-2'></a>
### 3.2 Compute the Cost

ATodo lo que tiene que hacer ahora es definir la función de pérdida que va a utilizar. Para este caso, como tenemos un problema de clasificación con 6 etiquetas, ¡una entropía cruzada categórica funcionará! 

<a name='ex-6'></a>
### Exercise 6 -  compute_cost

Implementa la función de coste que se muestra a continuación. 
- Es importante tener en cuenta que las entradas "`y_pred`" y "`y_true`" de [tf.keras.losses.categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/categorical_crossentropy) se espera que sean de la forma (número de ejemplos, num_classes). 

- `tf.reduce_mean` básicamente hace la suma sobre los ejemplos.
<p hidden>
cost = tf.reduce_mean( tf.keras.losses.categorical_crossentropy(
        tf.transpose(labels),
        tf.transpose(logits),from_logits=True))
<\p>

In [ ]:
# GRADED FUNCTION: compute_cost 

def compute_cost(logits, labels):
    """
    Calcula el coste
    
    Argumentos:
    logits -- salida de la propagación hacia delante (salida de la última unidad LINEAL), de forma (6, num_examples)
    labels -- vector de etiquetas "verdaderas", de la misma forma que Z3
    
    Devuelve:
    cost - Tensor de la función de coste
    """
    
    #(1 line of code)
    # cost = ...
    # YOUR CODE STARTS HERE  

    
    # YOUR CODE ENDS HERE
    return cost

In [ ]:
def compute_cost_test(target, Y):
    pred = tf.constant([[ 2.4048107,   5.0334096 ],
             [-0.7921977,  -4.1523376 ],
             [ 0.9447198,  -0.46802214],
             [ 1.158121,    3.9810789 ],
             [ 4.768706,    2.3220146 ],
             [ 6.1481323,   3.909829  ]])
    minibatches = Y.batch(2)
    for minibatch in minibatches:
        result = target(pred, tf.transpose(minibatch))
        break
        
    print(result)
    assert(type(result) == EagerTensor), "Use the TensorFlow API"
    assert (np.abs(result - (0.25361037 + 0.5566767) / 2.0) < 1e-7), "Test does not match. Did you get the mean of your cost functions?"

    print("\033[92mAll test passed")

compute_cost_test(compute_cost, new_y_train )

**Expected output**
```
tf.Tensor(0.4051435, shape=(), dtype=float32)
```

<a name='3-3'></a>
### 3.3 - Entrenar el modelo

Hablemos de los optimizadores. Especificarás el tipo de optimizador en una línea, en este caso `tf.keras.optimizers.Adam` (aunque puedes usar otros como SGD), y luego lo llamarás dentro del bucle de entrenamiento. 

Fíjate en la función `tape.gradient`: permite recuperar las operaciones registradas para la diferenciación automática dentro del bloque `GradientTape`. Luego, al llamar al método del optimizador `apply_gradients`, se aplicarán las reglas de actualización del optimizador a cada parámetro entrenable. Al final de esta tarea, encontrarás una documentación que explica esto con más detalle, pero por ahora, una simple explicación será suficiente. ;) 


Aquí deberías tomar nota de un importante paso extra que se ha añadido al proceso de entrenamiento por lotes: 

- `tf.Data.dataset = dataset.prefetch(8)` 

Lo que esto hace es prevenir un cuello de botella en la memoria que puede ocurrir cuando se lee del disco. La función `prefetch()` aparta algunos datos y los mantiene listos para cuando se necesiten. Lo hace creando un conjunto de datos fuente a partir de sus datos de entrada, aplicando una transformación para preprocesar los datos, y luego iterando sobre el conjunto de datos el número especificado de elementos a la vez. Esto funciona porque la iteración es en flujo, por lo que los datos no necesitan caber en la memoria. 

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implementa una red neuronal tensorflow de tres capas: LINEAR->RELU->LINEAL->RELU->LINEAL->SOFTMAX.
    
    Argumentos:
    X_train -- conjunto de entrenamiento, de forma (tamaño de entrada = 12288, número de ejemplos de entrenamiento = 1080)
    Y_train -- conjunto de prueba, de forma (tamaño de salida = 6, número de ejemplos de entrenamiento = 1080)
    X_test -- conjunto de entrenamiento, de forma (tamaño de entrada = 12288, número de ejemplos de entrenamiento = 120)
    Y_test -- conjunto de prueba, de forma (tamaño de salida = 6, número de ejemplos de prueba = 120)
    learning_rate -- tasa de aprendizaje de la optimización
    num_epochs -- número de épocas del bucle de optimización
    minibatch_size -- tamaño de un minibatch
    print_cost -- Verdadero para imprimir el coste cada 10 épocas
    
    Devuelve:
    parameters -- parámetros aprendidos por el modelo. Pueden ser utilizados para predecir.
    """
    
    costs = []                                        # To keep track of the cost
    train_acc = []
    test_acc = []
    
    # Initialize your parameters
    #(1 line)
    parameters = initialize_parameters()

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    optimizer = tf.keras.optimizers.Adam(learning_rate)
    
    # El CategoricalAccuracy registrará la precisión de este problema multiclase
    test_accuracy = tf.keras.metrics.CategoricalAccuracy()
    train_accuracy = tf.keras.metrics.CategoricalAccuracy()
    
    dataset = tf.data.Dataset.zip((X_train, Y_train))
    test_dataset = tf.data.Dataset.zip((X_test, Y_test))
    
    # Podemos obtener el número de elementos de un conjunto de datos utilizando el método de cardinalidad
    m = dataset.cardinality().numpy()
    
    minibatches = dataset.batch(minibatch_size).prefetch(8)
    test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)
    #X_train = X_train.batch(minibatch_size, drop_remainder=True).prefetch(8)# <<< extra step    
    #Y_train = Y_train.batch(minibatch_size, drop_remainder=True).prefetch(8) # loads memory faster 

    # Do the training loop
    for epoch in range(num_epochs):

        epoch_cost = 0.
        
        #necesitamos reiniciar el objeto para empezar a medir desde 0 la precisión en cada época
        train_accuracy.reset_states()
        
        for (minibatch_X, minibatch_Y) in minibatches:
            
            with tf.GradientTape() as tape:
                # 1. predict
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)

                # 2. loss
                minibatch_cost = compute_cost(Z3, tf.transpose(minibatch_Y))

            # Acumulamos la precisión de todos los lotes
            train_accuracy.update_state(minibatch_Y, tf.transpose(Z3))
            
            trainable_variables = [W1, b1, W2, b2, W3, b3]
            grads = tape.gradient(minibatch_cost, trainable_variables)
            optimizer.apply_gradients(zip(grads, trainable_variables))
            epoch_cost += minibatch_cost
        
        # Dividimos el coste de la época entre el número de muestras
        epoch_cost /= m

        # Print the cost every 10 epochs
        if print_cost == True and epoch % 10 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            print("Train accuracy:", train_accuracy.result())
            
            # Evaluamos el conjunto de pruebas cada 10 épocas para evitar la sobrecarga computacional
            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_accuracy.update_state(minibatch_Y, tf.transpose(Z3))
            print("Test_accuracy:", test_accuracy.result())

            costs.append(epoch_cost)
            train_acc.append(train_accuracy.result())
            test_acc.append(test_accuracy.result())
            test_accuracy.reset_states()


    return parameters, costs, train_acc, test_acc

In [ ]:
parameters, costs, train_acc, test_acc = model(new_train, new_y_train, new_test, new_y_test, num_epochs=100)

**Expected output**

```
Cost after epoch 0: 0.057612
Train accuracy: tf.Tensor(0.17314816, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.24166666, shape=(), dtype=float32)
Cost after epoch 10: 0.049332
Train accuracy: tf.Tensor(0.35833332, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.3, shape=(), dtype=float32)
...
```
Numbers you get can be different, just check that your loss is going down and your accuracy going up!

In [ ]:
# Plot the cost
plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations (per fives)')
plt.title("Learning rate =" + str(0.0001))
plt.show()


In [ ]:
# Plot the train accuracy
plt.plot(np.squeeze(train_acc))
plt.ylabel('Train Accuracy')
plt.xlabel('iterations (per fives)')
plt.title("Learning rate =" + str(0.0001))
# Plot the test accuracy
plt.plot(np.squeeze(test_acc))
plt.ylabel('Test Accuracy')
plt.xlabel('iterations (per fives)')
plt.title("Learning rate =" + str(0.0001))
plt.show()


**Felicitaciones.** Has llegado al final de esta tarea, y al final del material de esta semana. ¡Un trabajo increíble construyendo una red neuronal en TensorFlow 2.3! 

Aquí hay un resumen rápido de todo lo que acabas de lograr:

- Usaste `tf.Variable` para modificar tus variables
- Entrenado una red neuronal en un conjunto de datos de TensorFlow

Ahora eres capaz de aprovechar el poder de TensorFlow para crear cosas interesantes, más rápido. Muy bien. 

<a name='4'></a>
## 4 - Bibliography 

In this assignment, you were introducted to `tf.GradientTape`, which records operations for differentation. Here are a couple of resources for diving deeper into what it does and why: 

Introduction to Gradients and Automatic Differentiation: 
https://www.tensorflow.org/guide/autodiff 

GradientTape documentation:
https://www.tensorflow.org/api_docs/python/tf/GradientTape